In [ ]:
#creating table with summary of visitors and visits per step per variation

import pandas as pd

df_web_pt1 = pd.read_csv("df_final_web_data_pt_1.txt")
df_web_pt2 = pd.read_csv("df_final_web_data_pt_2.txt")
df_web = pd.concat([df_web_pt1, df_web_pt2], ignore_index=True)

df_exp = pd.read_csv("df_final_experiment_clients.txt")

df_merged = pd.merge(df_web, df_exp, on='client_id', how='inner')

df_filtered = df_merged[df_merged['Variation'].isin(['Test', 'Control'])]

page_summary = df_filtered.groupby(['Variation', 'process_step']).agg(
    unique_visits=('visit_id', 'nunique'),
    unique_visitors=('visitor_id', 'nunique')
).reset_index()

step_order = ['start', 'step_1', 'step_2', 'step_3', 'confirm']
page_summary['process_step'] = pd.Categorical(page_summary['process_step'], categories=step_order, ordered=True)
page_summary = page_summary.sort_values(['Variation', 'process_step'])

period_summary.to_csv("period_totals_summary.csv", index=False)
page_summary.to_csv("page_engagement_summary.csv", index=False)

In [ ]:
# summary and daily tables creation

import pandas as pd

df_web_pt1 = pd.read_csv("df_final_web_data_pt_1.txt")
df_web_pt2 = pd.read_csv("df_final_web_data_pt_2.txt")
df_web = pd.concat([df_web_pt1, df_web_pt2], ignore_index=True)

df_exp = pd.read_csv("df_final_experiment_clients.txt")

df_merged = pd.merge(df_web, df_exp, on='client_id', how='inner')

df_filtered = df_merged[df_merged['Variation'].isin(['Test', 'Control'])].copy()

df_filtered['date_time'] = pd.to_datetime(df_filtered['date_time'])
df_filtered['date'] = df_filtered['date_time'].dt.date

# table 1: summaries
variation_summary = df_filtered.groupby('Variation').agg(
    total_unique_visitors=('visitor_id', 'nunique'),
    total_unique_visits=('visit_id', 'nunique')
).reset_index()

# table 2: daily metrics
daily_summary = df_filtered.groupby(['date', 'Variation']).agg(
    daily_unique_visitors=('visitor_id', 'nunique'),
    daily_unique_visits=('visit_id', 'nunique')
).reset_index()

daily_summary = daily_summary.sort_values(by=['date', 'Variation'])

# Export
variation_summary.to_csv("variation_total_summary.csv", index=False)
daily_summary.to_csv("variation_daily_summary.csv", index=False)